In [5]:


import pandas
import os
import zipfile
import requests
import tqdm
import numpy as np
from sklearn import dummy, metrics, cross_validation, ensemble

import keras.models as kmodels
import keras.layers as klayers
import keras.backend as K
import keras


if not os.path.exists('./ml-1m'):
    url = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'
    response = requests.get(url, stream=True)
    total_length = response.headers.get('content-length')
    bar = tqdm.tqdm_notebook(total=int(total_length))
    with open('./ml-1m.zip', 'wb') as f:
        for data in response.iter_content(chunk_size=4096):
            f.write(data)
            bar.update(4096)
    zip_ref = zipfile.ZipFile('./ml-1m.zip', 'r')
    zip_ref.extractall('.')
    zip_ref.close()

In [7]:
# Read in the dataset, and do a little preprocessing,
# mostly to set the column datatypes.
users = pandas.read_csv('./ml-1m/users.dat', sep='::', 
                        engine='python', 
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings = pandas.read_csv('./ml-1m/ratings.dat', engine='python', 
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])
movies = pandas.read_csv('./ml-1m/movies.dat', engine='python',
                         sep='::', names=['movieid', 'title', 'genre']).set_index('movieid')
movies['genre'] = movies.genre.str.split('|')
users.age = users.age.astype('category')
users.gender = users.gender.astype('category')
users.occupation = users.occupation.astype('category')
ratings.movieid = ratings.movieid.astype('category')
ratings.userid = ratings.userid.astype('category')

In [8]:
# Count the movies and users
n_movies = movies.shape[0]
n_users = users.shape[0]

# Also, make vectors of all the movie ids and user ids. These are
# pandas categorical data, so they range from 1 to n_movies and 1 to n_users, respectively.
movieid = ratings.movieid.cat.codes.values
userid = ratings.userid.cat.codes.values


In [20]:
from keras.models import Sequential 
from keras import preprocessing
from keras.layers import Embedding
from keras.layers import Flatten, Dense, Dropout
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import Embedding, Flatten, Dense
from keras.layers import SimpleRNN
from keras.preprocessing import sequence
from keras.layers import Dense, Activation
from keras.layers import Input, LSTM, Dense

#Get labels
y = np.zeros((ratings.shape[0], 5))
y[np.arange(ratings.shape[0]), ratings.rating - 1] = 1

dout = 1.0

movie_input = keras.layers.Input(shape=[1])
movie_vec = keras.layers.Flatten()(keras.layers.Embedding(n_movies + 1, 32)(movie_input))
movie_vec = keras.layers.Dropout(dout)(movie_vec)

# Same thing for the users
user_input = keras.layers.Input(shape=[1])
user_vec = keras.layers.Flatten()(keras.layers.Embedding(n_users + 1, 32)(user_input))
user_vec = keras.layers.Dropout(dout)(user_vec)

# BUILDING THE LSTM
model = LSTM(64)
#Create input layer
input_vecs = keras.layers.merge([movie_vec, user_vec], mode='concat')
layer = Dense(32,activation='relu')(input_vecs)
layer = Dense(10,activation='relu')(layer)
layer = Dense(5,activation='relu')(layer)
final_layer = Dense(5,activation='softmax')(layer)

# Finally, we pull out the result!

model = kmodels.Model([movie_input, user_input], final_layer)
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
a_movieid, b_movieid, a_userid, b_userid, a_y, b_y = cross_validation.train_test_split(movieid, userid, y)
print(b_movieid.shape)
print(a_movieid.shape)
print(y)

history = model.fit([a_movieid, a_userid], a_y, 
                         nb_epoch=5, 
                         validation_data=([b_movieid, b_userid], b_y))



/Users/pascal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:31: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/Users/pascal/anaconda3/lib/python3.6/site-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


(250053,)
(750156,)
[[ 0.  0.  0.  0.  1.]
 [ 0.  0.  1.  0.  0.]
 [ 0.  0.  1.  0.  0.]
 ..., 
 [ 0.  0.  0.  0.  1.]
 [ 0.  0.  0.  1.  0.]
 [ 0.  0.  0.  1.  0.]]


/Users/pascal/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:50: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 750156 samples, validate on 250053 samples
Epoch 1/5


    32/750156 [..............................] - ETA: 3:31:39 - loss: 0.5006 - acc: 0.8000

...

141056/750156 [====>.........................] - ETA: 1:27 - loss: 0.4426 - acc: 0.8020

141280/750156 [====>.........................] - ETA: 1:27 - loss: 0.4426 - acc: 0.8020

141504/750156 [====>.........................] - ETA: 1:27 - loss: 0.4425 - acc: 0.8021

141856/750156 [====>.........................] - ETA: 1:27 - loss: 0.4425 - acc: 0.8020

KeyboardInterrupt: 

In [19]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined